In [2]:
import numpy as np

# ---------------------------
# Utilities
# ---------------------------
def _bisection_root(f, a=0.0, b=1.0, tol=1e-12, maxiter=200):
    fa, fb = f(a), f(b)
    if np.isnan(fa) or np.isnan(fb):
        raise ValueError("Function returned NaN at the interval endpoints.")
    if fa == 0:  # rare
        return a
    if fb == 0:  # (note: for PGFs, f(1)=0)
        return b
    if fa * fb > 0:
        raise RuntimeError("Bisection requires a sign change on [a,b].")
    for _ in range(maxiter):
        c = 0.5 * (a + b)
        fc = f(c)
        if abs(fc) < tol or (b - a) < tol:
            return c
        if fa * fc > 0:
            a, fa = c, fc
        else:
            b, fb = c, fc
    return 0.5 * (a + b)

# =========================================================
# A) Two–component Poisson mixture (mean enforced to R0)
# =========================================================

def p_lambda(R0: float, lam: float) -> float:
    """
    p_λ = (λ - R0) / (λ - R0/2), valid only for λ > R0.
    """
    if lam <= R0:
        raise ValueError("Require lam > R0.")
    return (lam - R0) / (lam - R0/2)

def pgf_poisson_mixture(s: float, R0: float, lam: float) -> float:
    """
    G(s) = pλ * exp((R0/2)(s-1)) + (1 - pλ) * exp(lam*(s-1))
    """
    p = p_lambda(R0, lam)
    return p * np.exp((R0/2) * (s - 1.0)) + (1.0 - p) * np.exp(lam * (s - 1.0))

def zero_prob_poisson_mixture(R0: float, lam: float) -> float:
    """
    P(X=0) = pλ * e^{-R0/2} + (1 - pλ) * e^{-lam}
    """
    p = p_lambda(R0, lam)
    return p * np.exp(-(R0/2)) + (1.0 - p) * np.exp(-lam)

def extinction_prob_poisson_mixture(R0: float, lam: float, tol=1e-12) -> float:
    """
    Solve q = G(q). If mean <= 1 (here the mean is R0 by construction), return 1.
    Otherwise, find the root in [0,1). Uses bisection on f(q)=G(q)-q.
    """
    if lam <= R0:
        raise ValueError("Require lam > R0.")
    if R0 <= 1.0:
        return 1.0
    f = lambda q: pgf_poisson_mixture(q, R0, lam) - q
    # Ensure sign change on [0, 1 - eps]
    eps = 1e-14
    return _bisection_root(f, a=0.0, b=1.0 - eps, tol=tol)

# =========================================================
# B) Negative Binomial offspring: NB(mean=R0, dispersion=r)
# =========================================================

def pgf_negbin(s: float, R0: float, r: float) -> float:
    """
    G(s) = (1 + (R0/r)*(1 - s))^{-r}
    """
    return (1.0 + (R0 / r) * (1.0 - s)) ** (-r)

def zero_prob_negbin(R0: float, r: float) -> float:
    """
    P(X=0) = (r / (r + R0))^r
    """
    return (r / (r + R0)) ** r

def extinction_prob_negbin(R0: float, r: float, tol=1e-12) -> float:
    """
    Solve q = G(q) with G from NB. If R0 <= 1, return 1; else root in [0,1).
    """
    if R0 <= 1.0:
        return 1.0
    f = lambda q: pgf_negbin(q, R0, r) - q
    eps = 1e-14
    return _bisection_root(f, a=0.0, b=1.0 - eps, tol=tol)




In [3]:
# ===========================
# Quick sanity-check examples
# ===========================
if __name__ == "__main__":
    R0 = 2.5

    for r in [0.025, 0.1, 1, 20]:
        p0 = zero_prob_negbin(R0, r)
        qext = extinction_prob_negbin(R0, r)
        print(f"[NegBin ] r={r:>4}:  P0={p0:.4f}  q_ext≈{qext:.4f}")
    for lam in [200, 40 ,10, 5]:
        p = p_lambda(R0, lam)
        p0 = zero_prob_poisson_mixture(R0, lam)
        qext = extinction_prob_poisson_mixture(R0, lam)
        print(f"[Mixture] λ={lam:>4}: pλ={p:.4f}  P0={p0:.4f}  q_ext≈{qext:.4f}")

[NegBin ] r=0.025:  P0=0.8910  q_ext≈0.9611
[NegBin ] r= 0.1:  P0=0.7219  q_ext≈0.8607
[NegBin ] r=   1:  P0=0.2857  q_ext≈0.4000
[NegBin ] r=  20:  P0=0.0948  q_ext≈0.1256
[Mixture] λ= 200: pλ=0.9937  P0=0.2847  q_ext≈0.6113
[Mixture] λ=  40: pλ=0.9677  P0=0.2773  q_ext≈0.5545
[Mixture] λ=  10: pλ=0.8571  P0=0.2456  q_ext≈0.4108
[Mixture] λ=   5: pλ=0.6667  P0=0.1932  q_ext≈0.2803
